# WellnessMate: Multi-Agent Health & Wellness Assistant

This project is my capstone submission for the Kaggle x Google AI Agents Intensive.

# WellnessMate: Multi-Agent Health & Wellness Assistant

This notebook implements **WellnessMate**, a multi-agent AI system that analyzes a person's wellness log (sleep, steps, mood, hydration, workouts, etc.) and generates simple, personalised recommendations for healthier habits.

Instead of just doing one-off data analysis, WellnessMate uses **AI agents** to:

- understand a user’s question (for example: *"How is my sleep affecting my mood?"*),
- decide which analyses are needed,
- run tools to compute trends and statistics from the data,
- and finally transform those numbers into human-friendly advice.

This project is my capstone submission for the **Kaggle x Google AI Agents Intensive**.  
The goal is to clearly demonstrate how to apply the course concepts (agents, tools, memory, observability, evaluation, etc.) to a realistic use case: **supporting better health and wellness routines**.


## Key AI Agents Concepts Demonstrated

In this capstone project, I apply multiple core concepts from the **Kaggle x Google AI Agents Intensive**. The project demonstrates at least **five** of the required agentic capabilities:

---

### **1. Multi-Agent System (Sequential + Loop Behaviour)**  
This project uses a structured multi-agent architecture:

- **PlannerAgent** – understands the user’s query and creates an analysis plan.
- **AnalyticsAgent** – runs the actual data analysis using tools (statistics, trends, correlations).
- **CoachAgent** – turns numerical results into personalised wellness recommendations.

The agents run in a **sequential pipeline**:

> User → PlannerAgent → AnalyticsAgent → CoachAgent → Final Output

I also show a simple **loop-style behaviour**, where the system can run multiple iterations to update or refine recommendations with new data.

---

### **2. Tools (Custom Tools + Code Execution)**  
The agents do not directly manipulate the raw data.  
Instead, they call a registry of **custom tools**, such as:

- `filter_by_date()`
- `compute_basic_stats()`
- `compute_correlations()`
- `recent_trend()`
- (optional) `generate_simple_plot()` using Python execution

This follows the tool-calling pattern taught in the course, where agents “take action” by calling tools rather than embedding all logic internally.

---

### **3. Sessions & Memory (Short-Term + Long-Term)**  
This system uses **two layers of memory**:

- **Session memory**  
  Stores the current conversation state and intermediate results.

- **Long-term memory**  
  Implemented as a simple JSON store that saves recurring wellness insights, such as patterns between sleep and mood.

This demonstrates how agents can use both **short-term session context** and **persistent long-term knowledge**.

---

### **4. Observability (Logging, Tracing, Metrics)**  
The system logs every major action into a shared **trace**, including:

- which agent ran,
- which tools were used,
- timestamps for each step.

I also track simple **metrics**:

- number of tool calls,
- runtime,
- number of generated recommendations.

This reflects the observability practices taught in the course (logging, tracing, metrics).

---

### **5. Agent Evaluation**  
I include lightweight evaluation checks:

- verifying dataset loading,
- checking that statistical tools produce expected outputs,
- ensuring that at least one recommendation is generated for normal queries.

This shows a basic but clear approach to **agent quality evaluation**, as required in the capstone.


## System Architecture Overview

### **1. High-Level Concept**
The idea behind WellnessMate is simple:

> “Given a user’s daily wellness logs (sleep, steps, mood, hydration, workouts),  
> how can an AI agent system understand the user’s questions, analyze their wellness patterns,  
> and suggest small, personalised improvements?”

Rather than using a single large script, the project uses multiple collaborating **agents**, each responsible for part of the workflow.

---

### **2. Agents in the System**

#### **1. PlannerAgent**
- Reads the user’s natural language question.  
- Determines what type of analysis is needed (trends, correlations, general overview, etc.).  
- Produces a simple **plan** describing which steps and tools should run.

#### **2. AnalyticsAgent**
- Takes the plan from the PlannerAgent.  
- Uses tools to analyze the wellness dataset:
  - compute basic statistics,
  - extract recent trends,
  - compute correlations between variables.

#### **3. CoachAgent**
- Takes the analysis results.  
- Converts numerical data into practical, personalised wellness recommendations.  
- Stores useful insights into **long-term memory** (Memory Bank style).

Together, these agents work in a **sequential pipeline**:
> Planner → Analytics → Coach

---

### **3. Tools Layer**
To follow the agentic pattern from the course, the system uses **tools** instead of embedding logic directly in agents.

Examples of custom tools:
- `compute_basic_stats()` – mean/min/max for each metric  
- `compute_correlations()` – checks relationships (e.g., sleep ↔ mood)  
- `recent_trend()` – extracts the last 7/14/30 days of data  
- `filter_by_date()` – optional date filtering  
- `generate_plot()` – code-based tool to create charts

Agents call these tools through a `tool_registry`, which imitates an MCP-style tool discovery mechanism.

---

### **4. Memory Layer**

#### **Session Memory**
- Tracks the current conversation.
- Stores partial results within a single agent run.

#### **Long-Term Memory**
- A simple JSON file (`wellness_memory.json`).
- Stores repeated or meaningful insights so the agent can re-use them across sessions.
- Demonstrates the concept of a lightweight **Memory Bank**.

---

### **5. Observability & Evaluation**

The system includes:

- **Logging & Tracing:**  
  Every agent action is recorded in a trace list with timestamps.

- **Metrics:**  
  For each run, the system records:
  - tool calls count,
  - runtime,
  - number of recommendations generated.

- **Evaluation Checks:**  
  Small tests verify:
  - the dataset loads,
  - the tools return expected outputs,
  - the agent produces suggestions.

These elements support debugging, transparency, and quality control, aligning with the course’s best practices for **agent observability**.

---

### **6. End-to-End Flow**

> User query  
> → PlannerAgent creates a plan  
> → AnalyticsAgent executes tools  
> → CoachAgent generates personalised recommendations  
> → Logs, memory updates, and metrics recorded  
> → Final wellness insights returned

This complete flow showcases how an AI agentic system can analyze wellness data and provide user-friendly, actionable insights.


## Dataset Loading & Tool Definitions

In this section, I load the wellness dataset and define the set of **custom tools** that the agents will use.

These tools follow the action-taking pattern taught in the course:  
instead of hardcoding logic inside the agent, each step of work is done by a separate tool function.

The tools include:

- filtering by date  
- computing basic statistics  
- computing correlations  
- extracting recent trends  
- (optional) generating plots

All tools are stored in a `tool_registry` so the agents can “call” them by name,  
similar to how AI agents interact with MCP-style tools.


In [ ]:
# --- Imports ---
import pandas as pd
import numpy as np
import json
import time
from pathlib import Path

# --- Load the dataset ---

# IMPORTANT:
# For now, we create a placeholder DataFrame so the notebook runs.
# Later, if you upload a real dataset to Kaggle, you can replace this part.

data_example = {
    "date": pd.date_range(start="2024-01-01", periods=30, freq="D"),
    "sleep_hours": np.random.uniform(4, 9, 30),
    "steps": np.random.randint(2000, 12000, 30),
    "mood": np.random.randint(1, 6, 30),  # 1 to 5 rating
    "water_glasses": np.random.randint(3, 10, 30),
    "workout_minutes": np.random.randint(0, 60, 30),
}

df = pd.DataFrame(data_example)

df.head()


## Custom Tools

These tools perform the core data operations.  
The agents (Planner, Analytics, Coach) will call these tools by name via a `tool_registry`.

Each tool is designed to be:

- simple  
- reusable  
- separable from agent logic  

This is exactly the tool-based design encouraged in the course.


In [ ]:
# --- CUSTOM TOOLS ---

def filter_by_date(df, start_date=None, end_date=None):
    data = df.copy()
    if start_date:
        data = data[data["date"] >= pd.to_datetime(start_date)]
    if end_date:
        data = data[data["date"] <= pd.to_datetime(end_date)]
    return data


def compute_basic_stats(df):
    metrics = ["sleep_hours", "steps", "mood", "water_glasses", "workout_minutes"]
    stats = {}

    for col in metrics:
        if col in df.columns:
            stats[col] = {
                "mean": float(df[col].mean()),
                "min": float(df[col].min()),
                "max": float(df[col].max()),
            }
    return stats


def compute_correlations(df):
    numeric_df = df.select_dtypes(include=[np.number])
    if numeric_df.shape[1] < 2:
        return {}
    corr = numeric_df.corr()
    return corr.to_dict()


def recent_trend(df, days=14):
    if df.empty:
        return df
    cutoff = df["date"].max() - pd.Timedelta(days=days)
    return df[df["date"] >= cutoff]


def generate_simple_plot(df):
    # Plot tool: visual code execution-style tool
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 4))
    plt.plot(df["date"], df["sleep_hours"])
    plt.title("Sleep Trend Over Time")
    plt.xlabel("Date")
    plt.ylabel("Sleep Hours")
    plt.tight_layout()
    plt.show()


# --- TOOL REGISTRY (MCP-like pattern) ---
tool_registry = {
    "filter_by_date": filter_by_date,
    "compute_basic_stats": compute_basic_stats,
    "compute_correlations": compute_correlations,
    "recent_trend": recent_trend,
    "generate_simple_plot": generate_simple_plot,
}

tool_registry


## Session Memory, Long-Term Memory & Logging

To make this system feel more like a real agent (not just a script),  
I add:

- **Session memory** – stores the current conversation and intermediate results.
- **Long-term memory** – a simple JSON file that stores important wellness insights
  across runs (Memory Bank style).
- **Logging / tracing** – records which agent or tool did what, and when.

These structures will be used by the agents in later sections to:

- remember past insights,
- debug what happened in a run,
- and support multi-turn conversations.


In [ ]:
# --- SESSION MEMORY ---

# This will store the messages / events for the current run.
session_history = []

# --- LONG-TERM MEMORY (Memory Bank style) ---

MEMORY_FILE = Path("wellness_memory.json")

def load_long_term_memory():
    """Load long-term wellness insights from disk (if any)."""
    if MEMORY_FILE.exists():
        try:
            return json.loads(MEMORY_FILE.read_text())
        except json.JSONDecodeError:
            # If file is corrupted, start fresh
            return []
    return []

def save_long_term_memory(memory):
    """Save long-term wellness insights to disk."""
    MEMORY_FILE.write_text(json.dumps(memory, indent=2))

long_term_memory = load_long_term_memory()

def add_insight_to_memory(text):
    """Append a new insight to the long-term memory."""
    entry = {
        "insight": text,
        "ts": time.time()
    }
    long_term_memory.append(entry)
    save_long_term_memory(long_term_memory)
    return entry

# --- LOGGING / TRACING ---

trace = []

def log_event(agent, message):
    """Record an event in the trace log."""
    event = {
        "ts": time.time(),
        "agent": agent,
        "message": message
    }
    trace.append(event)
    return event

# Quick sanity check
log_event("system", "Memory and logging initialised.")
long_term_memory[:3], trace[:1]


## Agent Definitions: Planner, Analytics, Coach

Now that the tools, memory, and logging are in place,  
I define three simple agents:

1. **PlannerAgent** – reads the user’s question and decides what kind of analysis is needed.
2. **AnalyticsAgent** – calls the tools to analyse the wellness data.
3. **CoachAgent** – turns the analysis into human-friendly wellness tips and stores insights in long-term memory.

These agents are not “smart” by themselves (no complex NLP here).  
Instead, they follow simple rule-based logic to simulate how an LLM-powered agent would:

- decide which tools to use,
- interpret data,
- and generate helpful recommendations.


## PlannerAgent – 
Reads the user’s question and decides what kind of analysis is needed.

In [ ]:
# --- PLANNER AGENT ---

def planner_agent(user_query: str):
    """
    Very simple planner:
    - Looks at the user's text.
    - Decides which analysis steps (tools) to run.
    - Returns a 'plan' dictionary.
    """
    log_event("planner", f"Received query: {user_query}")

    q = user_query.lower()
    plan = {"steps": []}

    # If the user mentions "trend" or "recent", focus on recent_trend + stats
    if "trend" in q or "recent" in q or "last" in q:
        plan["steps"].append("recent_trend")
        plan["steps"].append("basic_stats")

    # If the user talks about "relationship", "effect", "impact", use correlations
    if "relationship" in q or "effect" in q or "impact" in q or "correlat" in q:
        plan["steps"].append("correlations")

    # If they mention specific metrics
    focus_metrics = []
    if "sleep" in q:
        focus_metrics.append("sleep_hours")
    if "mood" in q:
        focus_metrics.append("mood")
    if "steps" in q or "walk" in q:
        focus_metrics.append("steps")

    if focus_metrics:
        plan["focus_metrics"] = focus_metrics

    # Default behaviour: if no specific keyword found, just do basic overview
    if not plan["steps"]:
        plan["steps"] = ["basic_stats"]

    log_event("planner", f"Plan created: {plan}")
    return plan


## AnalyticsAgent – 
Calls the tools to analyse the wellness data.

In [ ]:
# --- ANALYTICS AGENT ---

def analytics_agent(plan, df):
    """
    Runs the actual data analysis based on the plan.
    Uses the tools defined earlier.
    """
    log_event("analytics", f"Starting analytics with plan: {plan}")
    analysis_result = {}

    # Start with full dataset
    data_for_analysis = df.copy()

    # If the plan includes recent trend, reduce to last N days
    if "recent_trend" in plan.get("steps", []):
        data_for_analysis = tool_registry["recent_trend"](data_for_analysis, days=14)
        analysis_result["recent_data_rows"] = int(len(data_for_analysis))

    # Always compute basic stats if requested
    if "basic_stats" in plan.get("steps", []):
        analysis_result["basic_stats"] = tool_registry["compute_basic_stats"](data_for_analysis)

    # Compute correlations if requested
    if "correlations" in plan.get("steps", []):
        analysis_result["correlations"] = tool_registry["compute_correlations"](data_for_analysis)

    # Save which metrics we were focusing on (if any)
    if "focus_metrics" in plan:
        analysis_result["focus_metrics"] = plan["focus_metrics"]

    log_event("analytics", f"Analytics result keys: {list(analysis_result.keys())}")
    return analysis_result


## CoachAgent – 
Turns the analysis into human-friendly wellness tips and stores insights in long-term memory.

In [ ]:
# --- COACH AGENT ---

def coach_agent(analysis_result, user_query: str):
    """
    Turns numbers into simple wellness advice.
    Uses heuristic rules (if avg sleep < 7 -> suggest more sleep, etc.).
    Also writes a short summary to long-term memory.
    """
    log_event("coach", "Generating coaching tips")

    tips = []
    stats = analysis_result.get("basic_stats", {})
    focus = analysis_result.get("focus_metrics", [])

    sleep_stats = stats.get("sleep_hours")
    mood_stats = stats.get("mood")
    water_stats = stats.get("water_glasses")
    steps_stats = stats.get("steps")
    workout_stats = stats.get("workout_minutes")

    # Simple rules for sleep
    if sleep_stats:
        if sleep_stats["mean"] < 7:
            tips.append("Try to aim for at least 7 hours of sleep on most nights.")
        else:
            tips.append("Your average sleep duration looks good. Focus on keeping a consistent sleep schedule.")

    # Simple rules for mood
    if mood_stats:
        if mood_stats["mean"] <= 3:
            tips.append("Your average mood is on the lower side. Consider relaxing activities before bed and short breaks during the day.")
        else:
            tips.append("Your average mood looks fairly positive. Keep up habits that make you feel good.")

    # Simple rules for water
    if water_stats:
        if water_stats["mean"] < 8:
            tips.append("Try increasing your water intake towards 8 glasses per day.")
        else:
            tips.append("Your water intake looks good. Stay hydrated, especially on active days.")

    # Simple rules for steps
    if steps_stats:
        if steps_stats["mean"] < 6000:
            tips.append("Your daily steps are a bit low. Try adding short walks after meals or using stairs more often.")
        else:
            tips.append("Your step count is healthy. You can maintain this level or add variety with different types of movement.")

    # Simple rules for workout
    if workout_stats:
        if workout_stats["mean"] < 20:
            tips.append("You could benefit from a bit more intentional exercise. Even 20–30 minutes a day can help.")
        else:
            tips.append("You are getting a decent amount of workout time. Keep going and listen to your body's signals.")

    # If for some reason no tips were generated
    if not tips:
        tips.append("Data is limited or unclear, but try to maintain regular sleep, hydration, and gentle movement each day.")

    high_level_summary = "These suggestions are based on your recent wellness data and simple heuristic rules (not medical advice)."

    # Store a short summary in long-term memory
    memory_text = f"Generated {len(tips)} wellness tips for query: '{user_query[:50]}...'"
    add_insight_to_memory(memory_text)

    coaching = {
        "summary": high_level_summary,
        "tips": tips,
    }

    log_event("coach", f"Generated {len(tips)} tips")
    return coaching


## Quick Test

In [ ]:
# Quick test for the three agents together
test_query = "Show me recent trends and how my sleep and mood are doing."
plan = planner_agent(test_query)
analysis = analytics_agent(plan, df)
coaching = coach_agent(analysis, test_query)

plan, list(analysis.keys()), coaching["summary"], len(coaching["tips"])


## Orchestrator: End-to-End Agent Flow

In this section, I connect all the pieces into a single function:

> `run_wellness_agent(user_query)`

This function:

1. Records the start of a new session.
2. Calls **PlannerAgent** to create a plan.
3. Calls **AnalyticsAgent** to run the tools and analyse the data.
4. Calls **CoachAgent** to generate wellness tips.
5. Logs all important events into the trace.
6. Computes simple **metrics** (runtime, number of tips, etc.).
7. Returns a structured result that can be used in any interface (CLI, web app, etc.).

This is similar to how a deployed agent endpoint would work in a real system.


In [ ]:
# --- ORCHESTRATOR ---

def run_wellness_agent(user_query: str):
    """
    Run the full WellnessMate pipeline:
    - planner_agent -> analytics_agent -> coach_agent
    - track session history, trace, and basic metrics
    """
    # Use global variables defined earlier
    global trace
    trace = []  # reset trace for this run

    start_time = time.time()

    # Save user message in session memory
    session_history.append({"role": "user", "content": user_query})

    log_event("system", "Starting new wellness agent run")

    # 1. Planner
    plan = planner_agent(user_query)

    # 2. Analytics
    analysis_result = analytics_agent(plan, df)

    # 3. Coach
    coaching = coach_agent(analysis_result, user_query)

    end_time = time.time()
    runtime = end_time - start_time

    # --- METRICS ---
    tool_calls_estimate = 0
    # Very rough estimate: count keys that came from tools
    if "basic_stats" in analysis_result:
        tool_calls_estimate += 1
    if "correlations" in analysis_result:
        tool_calls_estimate += 1
    if "recent_data_rows" in analysis_result:
        tool_calls_estimate += 1

    metrics = {
        "runtime_seconds": round(runtime, 3),
        "estimated_tool_calls": tool_calls_estimate,
        "tips_count": len(coaching.get("tips", [])),
    }

    log_event("system", f"Run finished in {metrics['runtime_seconds']} seconds")

    # Final structured response
    response = {
        "user_query": user_query,
        "plan": plan,
        "analysis_result": analysis_result,
        "coaching": coaching,
        "metrics": metrics,
        "trace": trace,
    }
    return response


## Demo: Example Wellness Queries

In this section, I run the full `run_wellness_agent()` pipeline on a few example queries to show how the system behaves.

These examples simulate how a user might interact with the agent:

- asking about overall wellness,
- asking specifically about sleep and mood,
- asking about energy or activity.


In [ ]:
# --- DEMO RUNS ---

demo_queries = [
    "Give me an overview of my recent wellness.",
    "Show me recent trends and how my sleep is affecting my mood.",
    "How can I improve my energy based on my data?",
]

demo_results = []

for q in demo_queries:
    print("=" * 80)
    print("USER QUERY:", q)
    result = run_wellness_agent(q)
    demo_results.append(result)

    print("Plan:", result["plan"])
    print("Metrics:", result["metrics"])
    print("Summary:", result["coaching"]["summary"])
    print("Number of tips:", len(result["coaching"]["tips"]))
    print("Example tip:", result["coaching"]["tips"][0] if result["coaching"]["tips"] else "No tips")
    print()


In [ ]:
# Show the trace of the last run to demonstrate logging & tracing
print("Trace for last run:")
for event in demo_results[-1]["trace"]:
    print(f"[{event['agent']}] {event['message']}")


## Agent Evaluation

To make sure the agent system is not just "running" but also **behaving sensibly**,  
I include a small **evaluation section**.

The goal here is not to build a perfect, production-grade evaluation suite,  
but to demonstrate the core ideas from the AI Agents Intensive:

- sanity checks on the **data**,
- sanity checks on the **tools**,
- sanity checks on the **end-to-end agent behaviour**.

I use a few lightweight tests to verify that:

1. The dataset loads and is not empty.
2. The statistics tool returns values for key metrics (e.g. `sleep_hours`).
3. The full `run_wellness_agent()` pipeline can:
   - run without errors,
   - produce at least one wellness tip for a normal query.


In [ ]:
# --- AGENT EVALUATION FUNCTIONS ---

def test_dataset_not_empty(df):
    """Check that the dataset loaded and has at least one row."""
    return not df.empty

def test_basic_stats_contains_sleep(df):
    """Check that the basic stats tool returns an entry for sleep_hours."""
    stats = compute_basic_stats(df)
    return "sleep_hours" in stats and "mean" in stats["sleep_hours"]

def test_agent_produces_tips():
    """Check that the full pipeline produces at least one tip."""
    sample_query = "Give me an overview of my recent wellness."
    result = run_wellness_agent(sample_query)
    tips = result["coaching"].get("tips", [])
    return len(tips) > 0

def evaluate_system(df):
    """Run all evaluation tests and return a summary dict."""
    results = {}

    results["dataset_not_empty"] = test_dataset_not_empty(df)
    results["basic_stats_contains_sleep"] = test_basic_stats_contains_sleep(df)
    results["agent_produces_tips"] = test_agent_produces_tips()

    return results


In [ ]:
# --- RUN EVALUATION ---

evaluation_results = evaluate_system(df)
evaluation_results


In [ ]:
print("Agent Evaluation Results:")
for name, passed in evaluation_results.items():
    status = "PASS ✅" if passed else "FAIL ❌"
    print(f"- {name}: {status}")


## Prototype → Production: How This Agent Could Be Deployed

So far, this notebook shows **WellnessMate** as a prototype:  
everything runs inside one Jupyter notebook.

In a real-world scenario, the same agent system could be deployed so that
other applications (web apps, mobile apps, chat interfaces) can call it.

---

### 1. Single Entry Point: `run_wellness_agent()`

The key design choice in this project is the function:

```python
run_wellness_agent(user_query: str) -> dict


# Final Summary

**WellnessMate** is a complete multi-agent system built as part of the  
**Kaggle x Google AI Agents Intensive Capstone Project**.

This notebook demonstrates:

### ✅ 1. Multi-Agent System
- **PlannerAgent**  
- **AnalyticsAgent**  
- **CoachAgent**  
Each agent has a clear role and they work together in a sequential pipeline.

### ✅ 2. Tool Usage
Custom tools handle:
- statistics
- correlations
- trends
- plotting  
Tools are managed via a `tool_registry`, following agentic tool-calling patterns.

### ✅ 3. Memory & Sessions
- **Session memory** to track the conversation.
- **Long-term memory** using a JSON file (`wellness_memory.json`).

### ✅ 4. Observability
- Every major action is logged (agent name + message + timestamp).
- A trace is stored and printed to inspect the agent workflow.
- Simple runtime + tool call metrics computed for each query.

### ✅ 5. Agent Evaluation
Lightweight evaluation ensures:
- dataset is valid,
- tools work,
- the full agent pipeline generates useful tips.

### 🚀 Why This Matters
This project shows how AI agents can analyze personal wellness data  
and generate personalised insights using:
- tools,
- memory,
- agent orchestration,
- and human-readable outputs.

It also demonstrates how such a system can scale from a **prototype**  
(notebook) into a **production-ready backend agent** with minor adjustments.

---

### 🎉 End of Notebook
Thanks for reviewing my capstone project!
